In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt

import time
from datetime import datetime, timedelta

import pylib.conv_widget as cw
from pylib.tensorboardcmd import tensorboard_cmd

<!-- requirement: pylib/__init__.py -->
<!-- requirement: pylib/conv_widget.py -->
<!-- requirement: images/conv_movie.gif -->
<!-- requirement: images/portal-v.png -->
<!-- requirement: images/portal-h.png -->
<!-- requirement: images/conv_weights.png -->
<!-- requirement: images/conv_layer.png -->
<!-- requirement: images/CNN_schematic.png -->
<!-- requirement: pylib/mnist_dataset.py -->
<!-- requirement: pylib/tf_utils.py -->
<!-- requirement: pylib/caltech_images.py -->



# Convolutional Neural Networks


In the previous exercise, we flattened the MNIST images into vectors before feeding them into our model. Doing so, however, destroys information about shapes that are present in clusters of neighboring pixels. We can preserve this two dimensional information by using Convolutional Neural Networks (CNNs). 

CNNs are inspired by the [visual cortex](https://en.wikipedia.org/wiki/Visual_cortex), which is responsible for image processing in animals. Neurons in the visual cortex fire in response to stimuli that activate small sub-regions of an animal's visual field. We can therefore think of these neurons as filters that detect patterns across the animal's visual field. 

We will train a CNN to find a set of filters that can detect repeated shapes in MNIST images. Convolving the image with a filter produces another image with highlighted features called a feature map. 

## Convolutions


A convolution is a mathematical operation that is performed on two functions. For continuous functions, it is defined as:

$$ (f * g)(t) = \int_{-\infty}^{\infty} f(\tau)g(t-\tau) d\tau $$

For discrete functions, it is:

$$ (f * g)(t) = \sum_{m=-\infty}^{\infty} f[m] g[n-m] $$

A function that is a convolution of two other functions can be thought of as the overlapping area of the two original functions as one function is translated. 

![convolution](images/conv_movie.gif)

### Convolving images and filters


Play around with the widget below, which demonstrates the process of convolving a 7x7 pixel image with a 3x3 pixel filter. At each iteration, the value of one pixel in the image (denoted by the red square) is replaced by the average of the pixel values within the 3x3 filter. In convolutional neural networks, a weight is assigned to each pixel in the filter, and consequently, the "red pixel" is replaced by weighted sum of the surrounded pixels. 

In [ ]:
image = np.zeros((7, 7), dtype=float)
for i in range(1, 6):
    for j in range(6-i, 6):
        image[i][j] = 1

img_filter = np.ones((3, 3), dtype=float)
img_filter /= img_filter.size

titles = ('Image and kernel', 'Filtered image')
convwidget = cw.ConvWidget()
convwidget.interactive_convolution_demo(image, img_filter, vmax=1, titles=titles)

#### Strides


In the interactive widget, we perform the convolution at each pixel. However, we can choose to skip pixels. In Keras, we can set the number of pixels our filter moves at each step, or the **stride**. 

**Question**: How does the stride effect the dimension of the filtered image? Why would we want to change the stride?

In Keras' `Conv2D` function, we set the stride with `strides=[stride_width, stride_height, channels]`. For the MNIST data set, `strides=[k, k, 1]`. Since we are dealing with grayscale images, the number of channels is 1. If we had colored images, this value would be set to 3 (corresponding red, green, and blue values). 


#### Padding


Notice how the filter extends past the edge of the image while performing convolutions on some pixels. To deal with these edge cases, we typically **pad** the image or add more pixels to the perimeter of the image. This process ensures that the dimensions of the input image are the same as the dimensions of the output image. In the case of the MNIST data set, the image is padded with zeros (or white space). When might we run into problems when padding images?

### Example: Applying a simple edge filter on a handwritten digit


The magic behind the convolutional neural network lies in the fact that each filter learns to recognize certain patterns repeated throughout images. To build our intuition for why this is the case, we will consider two very simple filters $(1,-1)$ and $(1,-1)^T$ applied to an image consisting of only black and white pixels (with values of 1 or 0). The image consists of a black, horizontal line connected to a black, vertical line and roughly resembles a "7." See the cell below. We see from the two resulting convolved images that the filters picked out vertical and horizontal edges, respectively. 

In [ ]:
from scipy import signal as sg
ex_image = [[0, 0, 0, 0, 0],
            [1, 1, 1, 1, 0], # <---horizontal line of ones
            [0, 0, 0, 1, 0],
            [0, 0, 0, 1, 0],
            [0, 0, 0, 1, 0]]
                    # ^ vertical line of ones

print(sg.convolve(ex_image, [[1, -1]], "same")) #vertical edge
print(sg.convolve(ex_image, [[1], [-1]], "same")) #horizontal edge


If we apply the vertical and horizontal edge filters a handwritten 7, we get the following feature maps:
<table>
    <tr>
        <td> <img src="files/images/portal-v.png" style="width: 250px;"/> </td>
        <td> <img src="files/images/portal-h.png" style="width: 250px;"/> </td>
    </tr>
</table>

## Convolutional neural networks


Convolutional neural networks are comprised of a series of convolutional and max pooling layers, followed by a fully connected layer. Each of these will be explained in detail below. Let's first start off by repeating some of the steps we did in the previous notebook. 

In [ ]:
from pylib.tf_utils import mnist_train, mnist_test
from tensorflow.keras.utils import to_categorical as one_hot

X_train, y_train = mnist_train()
X_test, y_test = mnist_test()

y_train = one_hot(y_train)
y_test  = one_hot(y_test)

Now let's set our model parameters and create the input layer. This step is very similar to the one we did in the previous notebook, but now we assign the number of channels to `n_channels` and the filter size to `filt_size`. For the sake of simplicity, we will not use dropout in this model. 

In [ ]:
# Model parameters
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

n_classes = 10
n_channels = 1
filt_size = [5, 5] # 5x5 pixel filters

batch_size = 50
n_epochs = 5

### Convolutional layers


The purpose of the convolutional layer is to extract features from images. In this layer, a set of filters are convolved with some input to produce activation or feature maps. Then, the ReLU operation is applied to each pixel in the feature maps, converting negative pixel values to zero. During the training process, the CNN learns which filters activate when certain features are present. 

In the example below, we choose 16 filters and a stride of 1. 


In [ ]:
model = keras.models.Sequential()

model.add(keras.layers.Reshape([img_size, img_size, 1]))
model.add(keras.layers.Conv2D(16, filt_size, padding='same',
                              activation='relu'))

After training, our 16 filters will look similar to those shown in the figure on the left. Red represents positive values and blue represents negative values. If we convolve an image of a seven with these filters, we will get the feature maps in the figure on the right.

<table>
    <tr>
        <td> <img src="files/images/conv_weights.png" style="width: 450px;"/> </td>
        <td> <img src="files/images/conv_layer.png" style="width: 450px;"/> </td>
    </tr>
</table>

### Max pooling 

In image classification, the approximate locations of shapes and edges are more important than their exact locations. To add this type of flexibility into our model, we can downsample our convolved images. A variety of techniques for downsampling exist, but **max pooling** is a popular one. Max pooling partitions an image and outputs the maximum value of each partition. A commonly used partition or filter size is 2x2. In Keras, we set the stride of this filter to its dimension, such that pooling regions do not overlap with one another. 

**Question:** What happens to the resolution of an n by n image after this type of pooling?

In [ ]:
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2),
                                    padding='same'))

### Fully connected layer


Notice that a convolution-ReLU-pooling layer operates on a single feature map (one for each convolutional filter in the previous layer), where each map represents a high-level feature in the input image. The goal of our convolutional neural network is to (1) identify each feature in the image, (2) identify combinations of these features in the image, and then (3) use this information to classify the image. To this end, we eventually want to connect every neuron to one another in a **fully connected layer**. 

You can think of a fully connected layer as a layer in a basic or flat neural network. To create this layer in Keras, we reshape our 3-dimensional output into a vector and then apply the weights and biases. The size of the weights matrix is the total number of pixels ($14 \times 14 \times 16$) by the number of desired outputs. In this example, we choose 100 as the output size. 

You can have more than one fully connected layer, but the final activation function should be softmax. 

In [ ]:
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))

### Class prediction


Finally we pass this output through another layer with output size `n_classes`. This step is necessary to get our predictions.

In [ ]:
model.add(keras.layers.Dense(n_classes, activation='softmax'))

### Completing the model


Now let's train our model as before.

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.SGD(lr=0.1),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    epochs=n_epochs,
                    batch_size=batch_size,
                    validation_data=(X_test,y_test))

Let's use our model to make some predictions.

In [ ]:
def predict(idx):
    image = np.array([X_test[idx]])
    return np.argmax(model.predict(image)) 

for idx in range(4):
    actual = np.argmax(y_test[idx])
    plt.figure()
    plt.title("Predicted: %d, Actual: %d" % (predict(idx), actual))
    plt.imshow(X_test[idx].reshape((28,28)), cmap=plt.cm.gray_r)

## Summary


We go through the following steps when training a CNN:
1. Initialize the filters/weights and biases with random values.
2. Perform a series of convolution, ReLU, and pooling operations to produce feature maps.
3. Flatten the feature maps and apply weights and biases
4. Pass the output from the fully connected layer through a final, softmax layer
5. Calculate loss.
6. Minimize loss via gradient descent and adjust values of filters/weights and biases. 
7. Repeat previous steps with all training images.


## Pre-trained CNNs (`applications`)

In image processing with CNNs, it is commonplace to use pre-trained networks to save time training a deep CNN that may have $\mathcal{O}(10^7)$ parameters. Many successful CNNs can be used as feature extraction engines, utilizing the convolutional layers learned from a particular task (e.g. `ImageNet`) and replacing the classification layer at the top of the network. [Keras includes an `applications` module](https://keras.io/applications/) that includes many pre-trained networks.

In [ ]:
%%bash
wget -nv http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz -O ~/caltech101.tar.gz -nc
mkdir -p ~/caltech101
tar -xzf ~/caltech101.tar.gz -C ~/caltech101 --strip-components=1

In [ ]:
from pylib import caltech_images
TESTING = True # switch to False for full run
data = caltech_images.Caltech101('~/caltech101', TESTING)
dataset = data.generate_dataset()

In [ ]:
# minimum InceptionV3 image size is 139x139
# Caltech101 images have all different dimensions
inception = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(139, 139, 3))

Since we don't want to retrain the CNN, we should mark its layers as not trainable.

In [ ]:
for layer in inception.layers:
    layer.trainable = False

In [ ]:
batch_size = 128
n_batches = data.N_IMAGES // batch_size
x_train, y_train = dataset.repeat().batch(batch_size).make_one_shot_iterator().get_next()

In [ ]:
N_CLASSES = len(set(data.categories))

input_layer = keras.layers.Input(shape=(139, 139, 3),
                                   tensor=x_train)
x = inception(input_layer)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation='sigmoid')(x)
x = keras.layers.Dropout(.5)(x)
y = keras.layers.Dense(N_CLASSES, activation='softmax')(x)

model = keras.models.Model(inputs=input_layer, outputs=y)
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'],
              target_tensors=[y_train])

In [ ]:
# warning: this is quite slow on CPU
# BUMP EPOCHS to 50 for true training
EPOCHS=1 # 50
# model.fit(steps_per_epoch=n_batches, epochs=EPOCHS)

Another approach we could take would be to compute the latent vectors (the result of the images run through the Inception model) ahead of time and use those as our input features to the last few dense layers. This can decrease the computation time if the largest cost is pushing the images through a deep network like the Inception model.

## Exercise: Improving the network architecture


We can improve the model's accuracy by adding more layers and filters. Rewrite the model so that its architecture matches that in the figure below. Bonus points if you use dropout. 

![CNN schematic](images/CNN_schematic.png)

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*